# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')
MODEL=os.getenv('LLM_MODEL')

In [2]:
# For OpenAI after 1.0.0 (1.13.3 at the time of writing)
client = openai.OpenAI()

def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [3]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

Hola, me gustaría pedir una licuadora.


In [4]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

This sentence is in French.


In [5]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal: ¿Le gustaría pedir una almohada?
Informal: ¿Te gustaría pedir una almohada?


## Try it yourself!
Try some translations on your own!

### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [6]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [7]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (The language is French.): La performance du système est plus lente que d'habitude.
The system's performance is slower than usual.

시스템의 성능이 평소보다 느립니다. 

Original message (The language is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that do not light up.

Korean: 내 모니터에는 불이 들어오지 않는 픽셀이 있습니다. 

Original message (The language is Italian.): Il mio mouse non funziona
English: My mouse is not working

Korean: 내 마우스가 작동하지 않아요 

Original message (The language is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken.

Korean: 내 Ctrl 키가 고장났습니다. 

Original message (The language is Chinese.): 我的屏幕在闪烁
English: My screen is flickering.

Korean: 내 화면이 깜박거리고 있어요. 



## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.

In [8]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I hope this message finds you well. My name is Joe and I would like to bring your attention to the specifications of this particular standing lamp. 

Best Regards,
Joe


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [9]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

Here is the HTML table representation of the given JSON:

```html
<!DOCTYPE html>
<html>
<head>
    <title>Restaurant Employees</title>
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
        }
        th, td {
            border: 1px solid black;
            padding: 15px;
            text-align: left;
        }
    </style>
</head>
<body>
    <h2>Restaurant Employees</h2>
    <table>
        <tr>
            <th>Name</th>
            <th>Email</th>
        </tr>
        <tr>
            <td>Shyam</td>
            <td>shyamjaiswal@gmail.com</td>
        </tr>
        <tr>
            <td>Bob</td>
            <td>bob32@gmail.com</td>
        </tr>
        <tr>
            <td>Jai</td>
            <td>jai87@gmail.com</td>
        </tr>
    </table>
</body>
</html>
```

This HTML code creates a table with the title "Restaurant Employees". The table has two columns, "Name" and "Email", and three rows of data. Each row represents an employee with

In [10]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [11]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

"The girl with the black and white puppies has a ball."
No errors found.
"It's going to be a long day. Does the car need its oil changed?"
"There goes my freedom. They're going to bring their suitcases."
"You're going to need your notebook."
"That medicine affects my ability to sleep. Have you heard of the butterfly effect?"
"This phrase is to check ChatGPT for spelling ability."


In [12]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

I purchased this for my daughter's birthday because she constantly borrows mine from my room. Yes, adults also enjoy pandas. She carries it everywhere with her, and it's incredibly soft and adorable. However, one of the ears is slightly lower than the other, and I don't believe it was designed to be asymmetrical. It's somewhat small for the price I paid. I believe there might be larger options available for the same price. It arrived a day earlier than expected, allowing me to enjoy it myself before giving it to my daughter.


In [13]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style="color:red;font-weight:700;text-decoration:line-through;">Got </span><span style="color:red;font-weight:700;">I purchased </span>this for my <span style="color:red;font-weight:700;text-decoration:line-through;">daughter for her </span><span style="color:red;font-weight:700;">daughter's </span>birthday <span style="color:red;font-weight:700;text-decoration:line-through;">cuz </span><span style="color:red;font-weight:700;">because </span>she <span style="color:red;font-weight:700;text-decoration:line-through;">keeps taking </span><span style="color:red;font-weight:700;">constantly borrows </span>mine from my <span style="color:red;font-weight:700;text-decoration:line-through;">room.  </span><span style="color:red;font-weight:700;">room. </span>Yes, adults also <span style="color:red;font-weight:700;text-decoration:line-through;">like pandas too.  </span><span style="color:red;font-weight:700;">enjoy pandas. </span>She <span style="color:red;font-weight:700;text-decoration:line-through;">takes </span><span style="color:red;font-weight:700;">carries </span>it everywhere with her, and it's <span style="color:red;font-weight:700;text-decoration:line-through;">super </span><span style="color:red;font-weight:700;">incredibly </span>soft and <span style="color:red;font-weight:700;text-decoration:line-through;">cute.  One </span><span style="color:red;font-weight:700;">adorable. However, one </span>of the ears is <span style="color:red;font-weight:700;text-decoration:line-through;">a bit </span><span style="color:red;font-weight:700;">slightly </span>lower than the other, and I don't <span style="color:red;font-weight:700;text-decoration:line-through;">think that </span><span style="color:red;font-weight:700;">believe it </span>was designed to be asymmetrical. It's <span style="color:red;font-weight:700;text-decoration:line-through;">a bit </span><span style="color:red;font-weight:700;">somewhat </span>small for <span style="color:red;font-weight:700;text-decoration:line-through;">what </span><span style="color:red;font-weight:700;">the price </span>I <span style="color:red;font-weight:700;text-decoration:line-through;">paid for it though. </span><span style="color:red;font-weight:700;">paid. </span>I <span style="color:red;font-weight:700;text-decoration:line-through;">think </span><span style="color:red;font-weight:700;">believe </span>there might be <span style="color:red;font-weight:700;text-decoration:line-through;">other </span><span style="color:red;font-weight:700;">larger </span>options <span style="color:red;font-weight:700;text-decoration:line-through;">that are bigger </span><span style="color:red;font-weight:700;">available </span>for the same <span style="color:red;font-weight:700;text-decoration:line-through;">price.  </span><span style="color:red;font-weight:700;">price. </span>It arrived a day earlier than expected, <span style="color:red;font-weight:700;text-decoration:line-through;">so I got </span><span style="color:red;font-weight:700;">allowing me </span>to <span style="color:red;font-weight:700;text-decoration:line-through;">play with </span><span style="color:red;font-weight:700;">enjoy </span>it myself before <span style="color:red;font-weight:700;text-decoration:line-through;">I gave </span><span style="color:red;font-weight:700;">giving </span>it to my daughter.

## Try it yourself!
Try changing the instructions to form your own review.